<a href="https://colab.research.google.com/github/kauefs/alura/blob/%40/Projeto_Imersao_IA_Alura_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
[![GitHub](  https://img.shields.io/badge/-000000?logo=github&logoColor=FFFFFF)](https://github.com/kauefs/)
[![Medium](  https://img.shields.io/badge/-000000?logo=medium&logoColor=FFFFFF)](https://medium.com/@kauefs)
[![LinkedIn](https://img.shields.io/badge/-0077B5?logo=linkedin&logoColor=FFFFFF)](https://www.linkedin.com/in/kauefs/)
[![Python](  https://img.shields.io/badge/-3-4584B6?logo=python&logoColor=FFDE57&labelColor=4584B6&color=646464)](https://www.python.org/)
[![License]( https://img.shields.io/badge/Apache--2.0-D22128?style=flat&logo=apache&logoColor=CB2138&label=License&labelColor=6D6E71&color=D22128)](https://www.apache.org/licenses/LICENSE-2.0)

$$10\ de\ maio\ de\ 2024$$

# Projeto para a 2ª Edição da Imersão IA com [Alura](https://www.alura.com.br/) **&** [Google](https://gemini.google.com/)

## <ins>Rerranqueamento de Pesquisas na WikipédiA</ins>

Projeto inspirado em [exemplo](https://github.com/google-gemini/cookbook/blob/main/examples/Search_reranking_using_embeddings.ipynb) produzido pelo Google, usando a funcionalidade _Embedding_ da Inteligência Artificial **(**IA**)** **Gemini** do Google para rerranquear resultados de busca na WikipédiA.

_Embedding_ é uma técnica de Processamento de Linguagem Natural **(**PLN**)** que converte texto em vetores numéricos, capturando significado semântico **&** contexto, de forma que textos com conteúdos semelhantes apresentam _embeddings_ mais próximos, permitindo comparações textuais e o relacionamento entre textos, facilitando busca **&** classificação.

In [1]:
# @title Instalação & Configuração
# Instalando Pacotes:
!pip install wikipedia -q
# Importando Bibliotecas:
import  json
import  textwrap
import  wikipedia
import  numpy               as   np
import  google.generativeai as   genai
from    google.colab      import userdata
from   IPython.display    import Markdown, display
from wikipedia.exceptions import DisambiguationError, PageError
# Verificando Chave de Acesso à API:
try:
    api_key=userdata.get(  'api_key')
    genai.configure(api_key=api_key )
# Chave não Encontrada:
except userdata.SecretNotFoundError as e:
    print(f'''Chave-Secreta não encontrada!\n\n
            Deve-se criar uma chave chamada {api_key} no Colab\n\n
            Visite https://makersuite.google.com/app/apikey para criar uma chave de acesso à API\n\n
            Guarde-a na área de segredos no menu lateral esquerdo deste notebook (icone de chave)\n\n
            Nomeie a chave {api_key}''')
    raise e
# Chave sem Acesso ao NoteBook:
except userdata.NotebookAccessError as e:
    print(f'''É necessário dar acesso à chave {api_key} a este NoteBook
            para que ele possa acessar o sua conta no Gemini.''')
    raise e
# Erro Desconhecido:
except Exception as e:
    print(f'''Erro desconhecido; assegure-se de que a chave {api_key} está armazenada no Colab
            e que esteja válida, o que pode ser verificado em https://makersuite.google.com/app/apikey''')
    raise e
# Em não havendo erros,
# segue configuração para retorno das respostas em MarkDown para melhor legibilidade,
# bem como listagem dos modelos de IA disponíveis no Gemini:
else:
    def to_markdown(text):
        text = text.replace('•', '  *')
        return Markdown(textwrap.indent(text, '> ', predicate=lambda _:True))
# Listagem de Modelos Generativos:
    print('\nModelos Generativos:' )
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(m.name)
# Listagem de Modelos de Embedding:
    print('\nModelos de Embedding:')
    for m in genai.list_models():
        if 'embedContent'    in m.supported_generation_methods:
            print(m.name)


Modelos Generativos:
models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision

Modelos de Embedding:
models/embedding-001
models/text-embedding-004


### Função de Busca

Em atendimento ao mecanismo de busca, a função será estruturada da seguinte forma**:**
* Para a busca, será utilizado o método `wikipedia.search` para buscar tópicos relevantes;
* A partir desses tópicos, serão escolhidos os top `topics(int)` candidatos de cujas páginas o `gemini-pro` extrairá informações relevantes **(**neste momento, apenas extraindo informação, sem gerar conteúdo**)**;
* Histórico de buscas será mantido para evitar duplicidade.

In [2]:
# Construindo Função de Busca na WikePédia
# para buscar & resumir tópicos relevantes:
# https://pypi.org/project/wikipedia/
def wikipedia_search(search_queries:list[str])->list[str]:
    '''Pesquisa na WikipédiA para cada questão, retornando resumo das páginas relevantes.'''
    topics         = 3
    search_history =set()                                   # mantendo histórico de buscas
    search_urls    =[]
    mining_model   =genai.GenerativeModel('gemini-pro')
    summary_results=[]
    wikipedia.set_lang('pt')                               # definindo idioma da WikipédiA
# Termo de Busca:
    for query in search_queries:
        print(f'Buscando por "{query}"')
        search_terms = wikipedia.search(query)
# Termos Relacionados:
        print(f'Termos relacionados: {search_terms[:topics]}')
        for    search_term in search_terms[:topics]:        # seleção dos primeiros `n_topics` candidatos
            if search_term in search_history: continue      # verifica se o tópico já não foi buscado antes
# Retorno do termo buscado:
            print(f'Recuperando página: "{search_term}"')
            search_history.add(search_term)                 # adicionando ao histórico
# Usando o Gemini para extrair informação relevante:
            try:
                page     = wikipedia.page(search_term, auto_suggest=False)
                url      = page.url
                print(f'Fonte: {url}')
                search_urls.append(url)
                page     = page.content
                response = mining_model.generate_content(textwrap.dedent(f'''\
                            Extraindo informações relevantes sobre a pesquisa: {query}
                            Fonte:

                            {page}

                            Observação: sem resumo, apenas extração de informação relevante.'''))
# Verificando Citações Adicionais:
                urls = [url]
                if response.candidates[0].citation_metadata:
                    extra_citations = response.candidates[0].citation_metadata.citation_sources
                    extra_urls = [source.url for source in extra_citations]
                    urls.extend(extra_urls)
                    search_urls.extend(extra_urls)
                    print('Citações Adicionais:', response.candidates[0].citation_metadata.citation_sources)
                try                   :text = response.text
                except ValueError     :pass
                else                  :summary_results.append(text + '\n\nCom base em:\n  ' + ',\n  '.join(urls))
# Erro em caso de ambiquidade:
            except DisambiguationError:print(f'''Resultados quando procurando por "{search_term}"
                                        (originados da busca "{query}") foram ambíguos, ignorado…''')
# Erro em caso de página não encontrada:
            except PageError          :print(f'{search_term} não encontrou nenhuma página identificada, ignorando…')
# Imprimindo Fontes:
    print(f'Fontes:')
    for url in search_urls        :print('  ', url)
    return summary_results

In [3]:
# Testando o Processo:
teste = wikipedia_search(['O que é produto escalar?'])

Buscando por "O que é produto escalar?"
Termos relacionados: ['Produto escalar', 'Produto', 'Produto vetorial']
Recuperando página: "Produto escalar"
Fonte: https://pt.wikipedia.org/wiki/Produto_escalar
Recuperando página: "Produto"


/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Resultados quando procurando por "Produto"
                                        (originados da busca "O que é produto escalar?") foram ambíguos, ignorado…
Recuperando página: "Produto vetorial"
Fonte: https://pt.wikipedia.org/wiki/Produto_vetorial
Fontes:
   https://pt.wikipedia.org/wiki/Produto_escalar
   https://pt.wikipedia.org/wiki/Produto_vetorial


In [4]:
# Resultado da Busca:
for t in teste: display(to_markdown(t))

> **Definição de produto escalar:**
> 
> * Operação entre dois vetores que resulta em um número escalar (real).
> * Em álgebra linear, é o produto interno padrão do espaço euclidiano.
> 
> **Interpretação geométrica:**
> 
> * Produto das magnitudes euclidianas dos vetores e o cosseno do ângulo entre eles.
> * Dá a projeção escalar de um vetor na direção de outro.
> 
> **Interpretação algébrica:**
> 
> * Soma dos produtos dos componentes correspondentes dos vetores.
> 
> **Propriedades:**
> 
> * Comutativa: A ⋅ B = B ⋅ A
> * Distributiva em relação à soma vetorial: A ⋅ (B + C) = A ⋅ B + A ⋅ C
> * Multiplicação por escalar: (n1A) ⋅ (n2B) = (n1n2)(A ⋅ B)
> * Soma de quadrados: Aᵀ ⋅ A = a1² + a2² + ... + an²
> 
> Com base em:
>   https://pt.wikipedia.org/wiki/Produto_escalar

> **Produto Vetorial**
> 
> **Definição:**
> 
> * Operação binária entre dois vetores tridimensionais a e b, denotada por a × b.
> * Resulta em um vetor perpendicular a a e b, com magnitude igual à área do paralelogramo formado pelos vetores.
> 
> **Propriedades:**
> 
> * Anticomutativo: a × b = -b × a
> * Distributivo sobre a adição: a × (b + c) = a × b + a × c
> * Compatível com multiplicação escalar: (ra) × b = a × (rb) = r(a × b)
> * Não associativo, mas satisfaz a identidade de Jacobi: a × (b × c) + b × (c × a) + c × (a × b) = 0
> 
> **Aplicações:**
> 
> * Fórmula do operador rotacional
> * Descrição da força de Lorentz
> * Definição de torque e momento angular
> * Cálculo de normais de polígonos para computação gráfica
> 
> Com base em:
>   https://pt.wikipedia.org/wiki/Produto_vetorial

In [5]:
# @title Configuração do Modelo Generativo
model_name        =  'gemini-pro'
generation_config = {'candidate_count'  :1,
                     'temperature'      :.65,
                     'top_p'            :.95,
                     'top_k'            :3,
                     'stop_sequences'   :None,
                     'max_output_tokens':1024}
safety_settings   = {'HATE'      : 'BLOCK_NONE',
                     'HARASSMENT': 'BLOCK_NONE',
                     'SEXUAL'    : 'BLOCK_NONE',
                     'DANGEROUS' : 'BLOCK_NONE'}

Ao se passar uma função como argumento à ferramentas do modelo generativo, ele extrai um esquema da função para passar ao chamados da API, podendo retornar a chamar a função, mantendo para si uma referência à função para poder executá-la localmente mais tarde.

Observação**:** nesse caso, são permitidos apenas dados do tipo `int | float | str | dict` ou lista que contenha apenas esses tipos de dados permitidos.

In [6]:
# Passando a Função de Busca para o Modelo Generativo:
model             = genai.GenerativeModel(model_name       =     model_name,
                                          generation_config=generation_config,
                                          safety_settings  =    safety_settings,
                                          tools            =[wikipedia_search])

In [7]:
# @title Buscas Suplementares
# Suporte à busca original do usuário para cobrir a pesquisa de forma mais compreensiva.
# Passando Instruções ao Modelo:
instructions      = '''
Acesso a API da WikipédiA para responder buscas.
Gerar lista de possíveis resultados que possam responder a buscas.
Seja criativo usando frases chave da busca.
Gerar variedade de resultados fazendo perguntas relacionadas à busca para encontrar a melhor resposta.
Quanto mais perguntas gerar, melhores as changes de encontrar a resposta correta.

Exemplo:

user: Fale-me sobre os vencedores da Copa do Mundo de 2022.

function_call: wikipedia_search(['Qual time ganhou a Copa do Mundo de 2022?',
'Quem era o capitaão do time que ganhou a Copa do Mundo de 2022?',
'Que país sediou a Copa do Mundo de 2022?',
'Em qual estádio aconteceu o jogo da final da Copa do Mundo de 2022?',
'Copa do Mundo de 2022','Quem levantou o troféu da Copa do Mundo de 2022?'])

Use a lista de resumo de artigos retornada pela função de busca para responder à busca.

Aqui está a busca do usuário: {query}
                    '''

In [8]:
# @title Chamando a Função Automaticamente
# A sessão de chat do modelo generativo "conversará" com a função
# até retornar o resposta final ao usuário:
chat  =  model.start_chat(enable_automatic_function_calling=True)
query = 'Explique como os dinossauros desapareceram.'
res   =  chat.send_message(instructions.format(query=query))

Buscando por "Como os dinossauros desapareceram?"
Termos relacionados: ['Esturjão-branco', 'Triássico', 'Therapsida']
Recuperando página: "Esturjão-branco"
Fonte: https://pt.wikipedia.org/wiki/Esturj%C3%A3o-branco
Recuperando página: "Triássico"
Fonte: https://pt.wikipedia.org/wiki/Tri%C3%A1ssico
Recuperando página: "Therapsida"
Fonte: https://pt.wikipedia.org/wiki/Therapsida
Buscando por "Qual a teoria mais aceita sobre a extinção dos dinossauros?"
Termos relacionados: ['Extinção do Cretáceo-Paleogeno', 'Dinossauros', 'Extinção']
Recuperando página: "Extinção do Cretáceo-Paleogeno"
Fonte: https://pt.wikipedia.org/wiki/Extin%C3%A7%C3%A3o_do_Cret%C3%A1ceo-Paleogeno
Recuperando página: "Dinossauros"
Fonte: https://pt.wikipedia.org/wiki/Dinossauros
Recuperando página: "Extinção"
Fonte: https://pt.wikipedia.org/wiki/Extin%C3%A7%C3%A3o
Buscando por "O que causou a extinção dos dinossauros?"
Termos relacionados: ['Dinossauros', 'Extinção do Cretáceo-Paleogeno', 'Triássico']
Buscando por "Qua

In [9]:
# Resposta Final Formatada:
to_markdown(res.text)

> Os dinossauros desapareceram há cerca de 66 milhões de anos. A teoria mais aceita sobre sua extinção é a de que um asteroide ou cometa gigante colidiu com a Terra, causando mudanças climáticas e ambientais desastrosas. Isso levou à extinção de cerca de 75% das espécies da Terra, incluindo todos os dinossauros não aviários.

In [10]:
# Verificando Citações Adicionais:
res.candidates[0].citation_metadata or 'Nenhuma citação adicional encontrada.'

'Nenhuma citação adicional encontrada.'

In [11]:
# @title Verificando Passos no Histórico de Conversa
# Verificando  as trocas entre a Função e a API
# no histórico do chat:
for content in chat.history:
    part = content.parts[0]
    print(f'{content.role} -> ', end='')
    print(json.dumps(type(part).to_dict(part), indent=2))
    print('---' * 20)

user -> {
  "text": "\nAcesso a API da Wikip\u00e9diA para responder buscas.\nGerar lista de poss\u00edveis resultados que possam responder a buscas.\nSeja criativo usando frases chave da busca.\nGerar variedade de resultados fazendo perguntas relacionadas \u00e0 busca para encontrar a melhor resposta.\nQuanto mais perguntas gerar, melhores as changes de encontrar a resposta correta.\n\nExemplo: \n\nuser: Fale-me sobre os vencedores da Copa do Mundo de 2022.\n\nfunction_call: wikipedia_search(['Qual time ganhou a Copa do Mundo de 2022?',\n'Quem era o capita\u00e3o do time que ganhou a Copa do Mundo de 2022?',\n'Que pa\u00eds sediou a Copa do Mundo de 2022?',\n'Em qual est\u00e1dio aconteceu o jogo da final da Copa do Mundo de 2022?',\n'Copa do Mundo de 2022','Quem levantou o trof\u00e9u da Copa do Mundo de 2022?'])\n\nUse a lista de resumo de artigos retornada pela fun\u00e7\u00e3o de busca para responder \u00e0 busca.\n\nAqui est\u00e1 a busca do usu\u00e1rio: Explique como os dinossa

Pelo histórico do chat, é possível verificar os quatro passos de interação entre a função de busca e a API do Gemini**:**
1. Envio da pergunta do usuário;
2. Modelo responde chamando a função, adicionando perguntas relevantes;
3. Como a função está ativa para ser chamada automaticamente, ela é executada novamente, retornando lista de artigos ao modelo;
4. Seguindo as instruções de prompt, o modelo gera a resposta final com base nos artigos encontrados.

In [12]:
# @title Execução Manual da Função
# Apenas para Aprofundamento do Tema:
import     google.ai.generativelanguage as glm     # para código de baixo nível
chat   =       model.start_chat()
result =        chat.send_message(instructions.format(query=query))
# Passo 1: Inicialmente o modelo retorna uma FunctionCall:
fc     =      result.candidates[0].content.parts[0].function_call
fc     =    type(fc).to_dict(fc)
print(json.dumps(fc , indent=2))

{
  "name": "wikipedia_search",
  "args": {
    "search_queries": [
      "Como os dinossauros desapareceram?",
      "Qual \u00e9 a teoria mais aceita sobre a extin\u00e7\u00e3o dos dinossauros?",
      "O que causou a extin\u00e7\u00e3o dos dinossauros?",
      "Qual foi o impacto da extin\u00e7\u00e3o dos dinossauros na Terra?",
      "Quais s\u00e3o as evid\u00eancias que apoiam a teoria do impacto de um asteroide?"
    ]
  }
}


In [13]:
# Mostrando o Nome da Função:
fc['name']

'wikipedia_search'

In [14]:
# Passo 2: Chamando a Função com Argumentos Generativos:
summaries = wikipedia_search(**fc['args'])

Buscando por "Como os dinossauros desapareceram?"
Termos relacionados: ['Esturjão-branco', 'Triássico', 'Therapsida']
Recuperando página: "Esturjão-branco"
Fonte: https://pt.wikipedia.org/wiki/Esturj%C3%A3o-branco
Recuperando página: "Triássico"
Fonte: https://pt.wikipedia.org/wiki/Tri%C3%A1ssico
Recuperando página: "Therapsida"
Fonte: https://pt.wikipedia.org/wiki/Therapsida
Buscando por "Qual é a teoria mais aceita sobre a extinção dos dinossauros?"
Termos relacionados: ['Dinossauros', 'Extinção do Cretáceo-Paleogeno', 'Extinção']
Recuperando página: "Dinossauros"
Fonte: https://pt.wikipedia.org/wiki/Dinossauros
Recuperando página: "Extinção do Cretáceo-Paleogeno"
Fonte: https://pt.wikipedia.org/wiki/Extin%C3%A7%C3%A3o_do_Cret%C3%A1ceo-Paleogeno
Recuperando página: "Extinção"
Fonte: https://pt.wikipedia.org/wiki/Extin%C3%A7%C3%A3o
Buscando por "O que causou a extinção dos dinossauros?"
Termos relacionados: ['Dinossauros', 'Extinção do Cretáceo-Paleogeno', 'Triássico']
Buscando por "Q

In [15]:
# Passo 3: Enviando a Resposta ao Modelo:
response = chat.send_message(
    glm.Content(
        parts=[glm.Part(
            function_response = glm.FunctionResponse(
                name='wikipedia_search', response={'result':summaries}
                )
            )
        ]
    )
)
# Passo 4: Gerando Resposta Final:
to_markdown(response.text)

> Os dinossauros foram extintos há cerca de 66 milhões de anos, possivelmente devido ao impacto de um asteroide ou cometa na Terra. O impacto teria causado um inverno prolongado, bloqueando a luz solar e prejudicando a fotossíntese. A poeira e as partículas na atmosfera também teriam causado mudanças climáticas significativas, levando a um resfriamento da superfície da Terra. A interrupção da fotossíntese afetou toda a cadeia alimentar, contribuindo para a extinção dos dinossauros e de muitos outros organismos.

### Rerranqueamento dos Resultados da Busca

De forma a encontrar o resultado mais relevante obtido usando a API da WikipédiA, usar**-**se**-**á a API do Gemini para vetorizar **(**_Embedding_**)** os resultados das buscas a fim de calcular pontuação de similaridade **(**usando similaridade de cossenos ou produto escalar**)** entre os vetores **(**textos**)**, de forma a determinar o vetor mais relevante para responder ao usuário.

In [16]:
# @title Função Embedding
def get_embeddings(content:list[str])->np.ndarray:
    embeddings = genai.embed_content('models/embedding-001', content, 'SEMANTIC_SIMILARITY')
    embds      = embeddings.get('embedding', None)
    embds      = np.array(embds).reshape(len(embds),-1)
    return embds
# Função Produto Escalar:
def dot_product(a:np.ndarray,  b:np.ndarray):
    return (a @ b.T)
# Aplicando a Função Embedding:
search_res     = get_embeddings(summaries)
embedded_query = get_embeddings([query])
# Calculando Pontuação de Similaridade:
sim_value      = dot_product(search_res, embedded_query)
# Seleção do Melhor Candidato - np.argmax:
print(summaries[np.argmax(sim_value)])

O texto fornecido não contém informações sobre como os dinossauros desapareceram.

Com base em:
  https://pt.wikipedia.org/wiki/Therapsida


#### Similaridade com Embedding Hipotética de Documento **(**HyDE**)**

_Hypothetical Document Embeddings_ **(**HyDE**)** objetiva gerar resposta hipotética ao usuário usando conhecimento interno do Gemini para servir de base para cálculo da relevância dos demais resultados.

In [17]:
hyde_model = genai.GenerativeModel('gemini-pro')
res =  hyde_model.generate_content(f'''
Gere resposta hipotética para a busca do usuário usando seu próprio conhecimento.
Assuma que você sabe tudo sobre o tópico. Não use informação factual,
use substituições para completar sua resposta.

query: {query}
                                    ''')
to_markdown(res.text)

> No crepúsculo de uma era primitiva, quando titãs vagavam pela Terra, um cataclismo cósmico desencadeou um destino trágico para os reis que antes dominavam o planeta. Um objeto celestial colossal, um mensageiro do destino, rasgou os céus, deixando um rastro de destruição em seu caminho.
> 
> O impacto colossal desencadeou uma onda de choque devastadora que se propagou por todo o globo, reduzindo florestas a cinzas e lançando rochas no ar como confete. Nuvens de poeira e cinzas obscureceram o sol, mergulhando o mundo em uma escuridão eterna.
> 
> À medida que a poeira assentava, uma onda de calor escaldante varreu a Terra, queimando a vegetação restante e sufocando qualquer chance de sobrevivência. A atmosfera tornara-se um inferno tóxico, impróprio para os répteis gigantescos que antes prosperavam.
> 
> Um por um, os dinossauros sucumbiram ao cataclisma implacável. Alguns foram consumidos pelas chamas, enquanto outros pereceram por inanição ou pela falta de ar respirável. Apenas alguns poucos sobreviventes, adaptados a nichos especializados, conseguiram escapar do destino de seus gigantescos ancestrais.
> 
> E assim, o reinado dos dinossauros chegou ao fim, marcando o início de uma nova era, onde mamíferos e aves assumiram o manto do domínio.

In [18]:
# Embedding a resposta hipotética para comparar com os resultados da busca:
hyde_res  = get_embeddings([res.text])
# Calculando Pontuação de Similaridade para Ranquear os Resultados:
sim_value = dot_product(search_res, hyde_res)
# Verificando Valores:
sim_value

array([[0.75811477],
       [0.80441426],
       [0.74443895],
       [0.81285525],
       [0.80384239],
       [0.7952765 ],
       [0.84048431],
       [0.83033978],
       [0.78153983],
       [0.73533617]])

In [19]:
# # Seleção do Melhor Candidato - np.argmax:
to_markdown(summaries[np.argmax(sim_value)])

> O impacto da extinção dos dinossauros na Terra foi significativo e de longo alcance:
> 
> * **Criação de Oportunidades:** A extinção dos dinossauros não aviários abriu um nicho ecológico para outros organismos, permitindo a evolução de mamíferos, aves e outros grupos.
> * **Recuperação Lenta:** Demorou milhões de anos para o ecossistema se recuperar da perda de vida, com plantas e animais desenvolvendo novas adaptações para preencher os nichos deixados pelos dinossauros.
> * **Evolução dos Mamíferos:** Os mamíferos, anteriormente relegados a papéis menores, puderam evoluir para novas formas e ocupar habitats diversos, eventualmente dando origem aos humanos.
> * **Formação de Florestas:** A extinção de dinossauros herbívoros gigantes levou a um aumento na vegetação, incluindo florestas, que se espalharam por vastas áreas da Terra.
> * **Mudanças Climáticas:** A perda de dinossauros e outros grandes répteis levou a mudanças significativas no clima, pois esses animais desempenharam um papel na regulação dos níveis de dióxido de carbono na atmosfera.
> 
> Com base em:
>   https://pt.wikipedia.org/wiki/Hist%C3%B3ria_da_Terra

In [20]:
# Doações à WikipédiA:
wikipedia.donate()